This notebook will walk you through a use case to understand the concept of grounding. Grounding is used when you want the model to return reliable answers to a given question. Often times, the GPT models will need additional context to provide a response that does not hallucinate. There are a couple of methods to go about grounding. In this scenario, we will focus mainly on basic grounding within the prompt. In Challenge 3 and Challenge 4, you will see other applications of grounding by using an external knowledge base.

To understanding the benefits of grounding, the notebook will walk through an example to comprehend what happens before grounding and what happens after. 

The scenario is as follows: You are trying to figure out the right insurance plan to pick from your closest health care provider, Kaiser Permanente. As you do your research, you get stuck on picking between an individual or family plan. Use the GPT model and the completions API to help you find out the different types of individual and family plans that Kaiser Permanente offers.

In [4]:
# Import the needed modules
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Load your OpenAI credentials
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = "azure"
openai.api_version = "2022-12-01"

chat_model=os.getenv("MODEL_NAME")



In [5]:
def get_completion(prompt, model="text-davinci-003"):
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=0, 
        max_tokens = 500,
        top_p = 1.0,
    )
    return response.choices[0].text

# BEFORE GROUNDING

In [6]:
prompt = f"""
What are the types of individual and family plans kaiser Permanente offers? 
"""

model_response = get_completion(prompt, model=chat_model)
print(f"Response: {model_response}\n")


Response: 
Kaiser Permanente offers a variety of individual and family plans, including HMO, PPO, and EPO plans. HMO plans provide comprehensive coverage with a network of doctors and hospitals, while PPO plans offer more flexibility and access to out-of-network providers. EPO plans provide access to a select network of providers and hospitals. Additionally, Kaiser Permanente offers high-deductible health plans, health savings accounts, and short-term health plans.



It may not be so clear but the model is nto giving the right response. On the Kaiser Permanente website, there are different options for indiviual and family plans. We need to provide the model some context for it to reference when providing us a response. By adding in the right information for individual and family plans in the system message, we can get an accurate response. We can also tell the prompt to only answer questions based on the information provided in the text below. Let us see what the response is this time.

# AFTER GROUNDING

In [9]:

text = f"""
# You are an AI assistant that helps customers find information regarding the different insurance plans. Only answer questions using the information given below \

# We offer a variety of plans to help fit your needs and budget. All of them offer the same quality care, but the way they split the costs is different. Not all of these plans are available in all states. \

# Copay plans: Copay plans are the simplest. There is no deductible and you pay for care according to an easy-to-follow copay or coinsurance schedule. Your monthly premium is higher, but you’ll pay much less when you get care. \

# Deductible plans: With a deductible plan, your monthly premium is lower, but you’ll need to pay the full charges for most covered services until you reach a set amount known as your deductible. Then you’ll start paying less – a copay or coinsurance. Depending on your plan, some services, like office visits or prescriptions, may be available at a copay or coinsurance before you reach your deductible. \

# Virtual plans: Virtual plans are a type of deductible plan that have lower costs for preventive and primary care when the visits are via telehealth (virtual visits) versus in-person visits. With a Virtual plan, you have several ways to access high-quality care — for many health conditions — that’s both affordable and convenient. You’ll get to choose how you get care, by taking full advantage of our many no-cost virtual care options — while having access to in-person primary care whenever you need it. \

# HSA qualified plans: HSA-qualified deductible plans are deductible plans with a special feature. With this plan, you can set up a health savings account (HSA) to pay for health costs like copays, coinsurance, and deductible payments. And you won’t pay Federal income taxes on the money in this account. You can use your HSA anytime to pay for care, including some services that may not be covered by your plan, such as eyeglasses or adult dental. And if you have money left in your HSA at the end of the year, it will roll over for you to use the next year. \

# Catastrophic plans: These plans are available in some markets for people under age 30, and those over 30 with a hardship exemption. They have very high deductibles and low monthly premiums, and cover limited preventive benefits before the deductible. They might be a reasonable strategy to safeguard yourself against the worst-case events, such as becoming very ill or hurt. However, until the plan's yearly deductible is met, you are responsible for paying for most medical costs. \
 

# """


In [10]:
prompt = f"""
What are the types of individual and family plans kaiser Permanente offers based on the text delimited by triple backticks.
```{text}```
"""

model_response = get_completion(prompt, model=chat_model)
print(f"Response: {model_response}\n")

Response: 
Kaiser Permanente offers four types of individual and family plans: Copay plans, Deductible plans, Virtual plans, and HSA qualified plans. Catastrophic plans are also available in some markets.



We see clearly that the response has changed. The model is giving us a response based on the context we provided to it. The answer is both reliable and we have reduced the chances of model hallucinations happening.